In [ ]:
#CIFAR-10
import os
import argparse
import torch



import torchvision
import numpy as np
from modules import resnet, network, transform
# from evaluation import evaluation

from torch.utils import data
import copy
import gc
import matplotlib.pyplot as plt
from matplotlib import pyplot


In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu

gen_input_nc = 3
# load the generator of adversarial examples
pretrained_generator_path = 'netG_cc_yale-attack_epoch_300.pth'
# pretrained_generator_path = 'netG_cc_yale-attack2_epoch_30.pth'
pretrained_G = models_clu.Generator(gen_input_nc, 3).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
from scipy.misc import imread

import matplotlib.pyplot as plt

x = imread('./ExtendedYaleB/yaleB20/yaleB20_P04A+025E+00.pgm').astype(float)
#x = np.divide(x, np.max(x))
print(x)


plt.imshow(x)
plt.show()
plt.imsave('./ExtendedYaleB_adv/yaleB20_P04A+025E+00.jpeg', x)

In [ ]:
from PIL import Image

x = np.array(Image.open('./ExtendedYaleB/yaleB20/yaleB20_P04A+025E+00.pgm'))
x = np.divide(x, 255)

print(x)

plt.imshow(x, cmap='gray')
plt.show()
plt.imsave('./ExtendedYaleB_adv/test.jpeg', x, cmap='gray')

In [ ]:
# x = torch.from_numpy(x)
# perturbation = pretrained_G(x)     

# perturbation = torch.clamp(perturbation, -0.05, 0.05)
# adv_img = perturbation + x
# adv_img = torch.clamp(adv_img, 0, 1)

In [ ]:
# import cv2

# # x = cv2.cvtColor('./ExtendedYaleB/yaleB20/yaleB20_P04A+025E+00.pgm', cv2.COLOR_BGR2RGB)
# x = cv2.imread('./ExtendedYaleB/yaleB20/yaleB20_P04A+025E+00.pgm',-1) 
# x = np.array(x)
# x


In [ ]:
path = "ExtendedYaleB"
imageid2ground = {}
images_stored = []

persons = os.listdir(path)

for ground, person in enumerate(persons):
    co = 0
    images = os.listdir(path + '/' + person)
#     print(f'curr:{ground} person:{person}')
    for image in images:
#         print(f'curr:{ground} person:{person} co:{co}')
        if co > 10:
            break
#         with open(path + '/' + person + '/'+ image, "rb") as img_file:
#             img = img_file.read()

In [ ]:
dataset = torchvision.datasets.ImageFolder(
    root='./ExtendedYaleB',
    transform=transform.Transforms(size=(192)).test_transform,
)

data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=64,
        drop_last=True,
        num_workers=1,
)

In [ ]:
label_map = {}
co = 0

persons = os.listdir(path)

for ground, person in enumerate(persons):
    label_map[ground] = person
label_map

In [ ]:
from collections import defaultdict
import os

# file_nu = defaultdict(int)
def save(img, label, file_nu, adv= False):
    folder = label_map[label]
    root='./custom3'
    if adv:
        root = './custom_adv3'
    try:
        os.mkdir(f'{root}/{folder}')
    except:
        pass
        
    plt.imsave(f'{root}/{folder}/{file_nu[folder]}.jpeg', img, cmap='gray')
    file_nu[folder] += 1
    

In [ ]:
# # # create sub-folders
# # for j in d.keys():
# os.mkdir('./custom/yolo')
    

In [ ]:
def process_batch(images, labels):
    perturbation = pretrained_G(images.cuda())
    perturbation = torch.clamp(perturbation, -0.07, 0.07)
    adv_imgs = perturbation + images.cuda()
    adv_imgs = torch.clamp(adv_imgs, 0, 1)
    
    
    for i in range(len(images)):
        img = np.transpose(images[i], (1, 2, 0))
#         plt.imshow(img)
#         plt.show()
        try:
            adv_img = adv_imgs[i].detach().cpu().numpy() 
        except:
            pass
        
        adv_img = np.transpose(adv_img, (1, 2, 0))
#         plt.imshow(adv_img)
#         plt.show()
#         break
        
        save(img, int(labels[i]), file_nu)
    
        save(adv_img, int(labels[i]), file_nu_adv, adv=True)
        
    torch.cuda.empty_cache()
#     break

In [ ]:
# for ground, person in enumerate(persons):
#     co = 0
#     images = os.listdir(path + '/' + person)

file_nu = defaultdict(int)
file_nu_adv = defaultdict(int)
for step, (x, y) in enumerate(data_loader):
    torch.cuda.empty_cache()
    process_batch(x,y)
    torch.cuda.empty_cache()
#     break
        